In [1]:
import sys
sys.path.append('../../ml_utils')

import config as cfg
import data_utils as du
import autoencoder as aenc
import timeit

import keras
import sklearn
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'

Using TensorFlow backend.
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/administrator/anaconda3/envs/masterthesis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [2]:
print("Python: ", sys.version)
print("pandas: ", pd.__version__)
print("numpy: ", np.__version__)
print("Keras: ", keras.__version__)

Python:  3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
pandas:  0.25.3
numpy:  1.17.4
Keras:  2.2.4


### Load and clean the data

In [3]:
log_data = './../../shared/data/swissid_authorize_logs_april_to_sept_2019.csv'

start = timeit.default_timer()
df = pd.read_csv(filepath_or_buffer=log_data, header=0, sep='\t', names=cfg.complete_feature_list, index_col=None)
stop = timeit.default_timer()

print("Original data frame size: {}\n".format(df.shape))
print("Time: {} seconds\n".format(stop - start))

Original data frame size: (12417597, 31)

Time: 39.18794764500126 seconds



In [4]:
df = du.clean_data(df)

columns with NaN: loc_country_code
loc_country_code is fixed


In [5]:
features = ['label_nr', 'src_software_sub_type', 'src_operating_system_name', 'src_hardware_type', 'response_status_code', 
            'oidc_client_id', 'oidc_scopes', 'oidc_ui_locales', 'loc_city', 'loc_country_code', 'date_weekday']

excludes = list(set(df.columns.tolist()) - set(features))

In [6]:
reduced_df = du.reduce_features(df, excludes)

print("reduced data frame size: {}\n".format(reduced_df.shape))
#display(reduced_df)

reduced data frame size: (12204748, 11)



In [7]:
print(reduced_df['label_nr'].value_counts())

2    7412813
0    4769169
1      22766
Name: label_nr, dtype: int64


In [8]:
def create_model(optimizer='adam', activation='relu'):
    
    model = aenc.create_simple_auto_encoder(10, 4, activation)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae', 'accuracy'])

    return model

In [9]:
np.random.seed(7)

sample_size= 200000
exclude = ['label_nr']

# load dataset
X_df, y_df, anomaly_rate, encoder = du.build_normal_anomaly_sample_df(reduced_df, sample_size, exclude, True, True, False, True)
x_train, x_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [10]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


model = KerasClassifier(build_fn=create_model, 
                        epochs=20, 
                        batch_size=32, 
                        verbose=2, 
                        shuffle=True)

# define the grid search parameters
optimizer = ['RMSprop', 'Adadelta', 'Adam', 'Adamax']
activation = ['softmax', 'relu', 'tanh', 'sigmoid']
batch_size = [32, 64, 128]
epochs = [50, 100, 150]

param_grid = dict(optimizer=optimizer, batch_size=batch_size, epochs=epochs, activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

start = timeit.default_timer()
grid_result = grid.fit(x_train, x_train)
stop = timeit.default_timer()

# summarize results
print("Time: {} minutes\n".format((stop - start)/60))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/150
 - 3s - loss: 0.8958 - mean_absolute_error: 0.5039 - acc: 0.5176
Epoch 2/150
 - 3s - loss: 0.8537 - mean_absolute_error: 0.4805 - acc: 0.7114
Epoch 3/150
 - 3s - loss: 0.8346 - mean_absolute_error: 0.4782 - acc: 0.7500
Epoch 4/150
 - 3s - loss: 0.8320 - mean_absolute_error: 0.4757 - acc: 0.7351
Epoch 5/150
 - 3s - loss: 0.8284 - mean_absolute_error: 0.4746 - acc: 0.7811
Epoch 6/150
 - 3s - loss: 0.8236 - mean_absolute_error: 0.4741 - acc: 0.8342
Epoch 7/150
 - 3s - loss: 0.8218 - mean_absolute_error: 0.4734 - acc: 0.8491
Epoch 8/150
 - 3s - loss: 0.8214 - mean_absolute_error: 0.4731 - acc: 0.8476
Epoch 9/150
 - 3s - loss: 0.8211 - mean_absolute_error: 0.4730 - acc: 0.8433
Epoch 10/150
 - 3s - loss: 0.8209 - mean_absolute_error: 0.4730 - acc: 0.8433
Epoch 11/150
 - 3s - loss: 0.8208 - mean_absolute_error: 0.4729 - acc: 0.8436
Epoch 12/150
 - 3s - loss: 0.82

Epoch 99/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8546
Epoch 100/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8515
Epoch 101/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8514
Epoch 102/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8514
Epoch 103/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8552
Epoch 104/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8520
Epoch 105/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8520
Epoch 106/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8512
Epoch 107/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8538
Epoch 108/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8519
Epoch 109/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8529
Epoch 110/150
 - 3s - loss: 0.8149 - mean_absolute_error: 0.4730 - acc: 0.8533
Epoch 111/150
 - 3s - loss: 0.8149 - mean_absolute_er